In [87]:
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score
from sklearn.feature_extraction.text import CountVectorizer
from xgboost import XGBClassifier
import pandas as pd
import matplotlib.pyplot as pl
import io



Läs in data

In [57]:
#os.chdir("F:\\03-github\\turnusol\\turnusol\\experiments")
data = pd.read_csv("..\\data\\dataset.csv").dropna()

data.sample(5)

,text,humor
4834,What do you call a pickle that draws? a dillus...,True
179168,Right-wing media group that backs limbaugh sud...,False
50466,10 ways to stay in shape while traveling,False
71360,Eight things you probably didn't know about mi...,False
37901,What would happen if you shot donald trump in ...,True


Lek lite

In [129]:
cv = CountVectorizer(max_features=2000, encoding="utf-8",
      ngram_range = (1,3),
      token_pattern = "[A-Za-z_][A-Za-z\d_]*")
X = cv.fit_transform(data.text).toarray()
y = data['humor']
X_train, X_test, y_train, y_test = train_test_split(X, y,
      test_size=0.2,
      random_state=0)
count_df = pd.DataFrame(X_train, columns=cv.get_feature_names_out())

#count_df['etiket'] = y_train


In [130]:
# fit model to training data

model = XGBClassifier(verbosity=1)
model.fit(X_train, y_train)


C:\Users\menbo\anaconda3\envs\scientificProject39\lib\site-packages\xgboost\sklearn.py:1224: UserWarning: The use of label encoder in XGBClassifier is deprecated and will be removed in a future release. To remove this warning, do the following: 1) Pass option use_label_encoder=False when constructing XGBClassifier object; and 2) Encode your labels (y) as integers starting with 0, i.e. 0, 1, 2, ..., [num_class - 1].
  warnings.warn(label_encoder_deprecation_msg, UserWarning)


[21:38:09] WARNING: ..\src\learner.cc:1115: Starting in XGBoost 1.3.0, the default evaluation metric used with the objective 'binary:logistic' was changed from 'error' to 'logloss'. Explicitly set eval_metric if you'd like to restore the old behavior.


XGBClassifier(base_score=0.5, booster='gbtree', colsample_bylevel=1,
              colsample_bynode=1, colsample_bytree=1, enable_categorical=False,
              gamma=0, gpu_id=-1, importance_type=None,
              interaction_constraints='', learning_rate=0.300000012,
              max_delta_step=0, max_depth=6, min_child_weight=1, missing=nan,
              monotone_constraints='()', n_estimators=100, n_jobs=16,
              num_parallel_tree=1, predictor='auto', random_state=0,
              reg_alpha=0, reg_lambda=1, scale_pos_weight=1, subsample=1,
              tree_method='exact', validate_parameters=1, verbosity=1)

In [131]:

# make predictions for test data

y_pred = model.predict(X_test)
#predictions = [value for value in y_pred]

# evaluate predictions

accuracy = accuracy_score(y_test, y_pred)
print("Accuracy: %.2f%%" % (accuracy * 100.0))

Accuracy: 91.70%


In [132]:
cv_test = CountVectorizer(vocabulary=cv.get_feature_names_out())
simple_test = pd.read_csv("..\\data\\simple_test.csv")
fitted = cv_test.fit_transform(simple_test.title)
new_pred = pd.DataFrame(model.predict(fitted))
new_pd = pd.concat([simple_test, new_pred],axis=1)
new_pd.columns = ["title", "result"]
new_pd["result"].value_counts()

True     60
False    40
Name: result, dtype: int64

In [133]:
new_pd

,title,result
0,My wife told me to stop impersonating a flamin...,True
1,I went to buy some camo pants but couldn’t fin...,False
2,"I failed math so many times at school, I can’t...",False
3,"I used to have a handle on life, but then it b...",True
4,I was wondering why the frisbee kept getting b...,True
...,...,...
95,"‘I have a split personality,’ said Tom, being ...",False
96,My teachers told me I’d never amount to much b...,True
97,Will glass coffins be a success? Remains to be...,False
98,Did you hear about the guy whose whole left si...,True
